In [ ]:
#sudo docker run --gpus all -p 8888:8888 -it --rm tensorflow/tensorflow:latest-gpu-jupyter
#jupyter notebook

#!pip install lambeq

In [1]:
from lambeq import BobcatParser, TreeReader, TreeReaderMode, spiders_reader, cups_reader, stairs_reader
from lambeq import TensorAnsatz, SpiderAnsatz, MPSAnsatz, AtomicType
from discopy import Dim
from classic_pipeline import *
from utilities import *

/home/adriano22_/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#define atomic-types

N = AtomicType.NOUN
S = AtomicType.SENTENCE
C = AtomicType.CONJUNCTION
P = AtomicType.PUNCTUATION
NP = AtomicType.NOUN_PHRASE
PP = AtomicType.PREPOSITIONAL_PHRASE

In [3]:
#parser declaration

bobcat_parser = BobcatParser(verbose = "progress")
spider_parser = spiders_reader
cups_parser = cups_reader
stairs_parser = stairs_reader
tree_parser = TreeReader(mode=TreeReaderMode.RULE_ONLY)

In [4]:
#ansatze declaration

tensor_ansatz = TensorAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)})

In [ ]:
#data-extraction for classic pipeline

pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/CPN_edited.csv", "n")
eval_labels, eval_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/ePurse_edited.csv", "n")

In [6]:
save_data("train_data.txt", train_labels, train_circuits)
save_data("test_data.txt", test_labels, test_circuits)
save_data("eval_data.txt", eval_labels, eval_circuits)

#train_labels, train_circuits = load_data("train_data.txt")
#test_labels, test_circuits = load_data("test_data.txt")
#eval_labels, eval_circuits = load_data("eval_data.txt")

In [28]:
temp_dict = {}
for diagram in train_circuits:
    key = len(diagram)
    if temp_dict.get(key) == None:
        temp_dict[key] = 1
    else:
        temp_dict[key] += 1
        
train_dict = dict(sorted(temp_dict.items()))
for key, val in train_dict.items():
    print(f"{key}   {val}")

18   1
20   2
21   3
23   2
24   1
25   2
26   1
27   2
29   2
30   2
31   5
32   4
33   3
34   2
35   2
36   3
37   3
38   3
39   2
40   4
42   2
43   2
44   2
45   6
46   1
47   2
48   8
49   4
50   2
51   3
52   4
53   1
54   3
57   4
58   4
59   2
60   2
61   2
62   1
65   3
66   1
68   1
70   1
72   2
73   1
74   1
75   3
76   2
79   2
82   1
83   2
89   1
90   1
94   2
95   1
98   2
100   1
101   1
102   1
103   2
104   1
106   1
107   1
113   1
115   1
118   3
122   1
126   1
127   2
132   1
140   1
156   4
161   1
170   1
171   2
180   1
181   2
187   1
189   1
195   2
197   1
218   1
276   1


In [8]:
temp_dict = {}
for diagram in test_circuits:
    key = len(diagram)
    if temp_dict.get(key) == None:
        temp_dict[key] = 1
    else:
        temp_dict[key] += 1
        
test_dict = dict(sorted(temp_dict.items()))

In [9]:
temp_dict = {}
for diagram in eval_circuits:
    key = len(diagram)
    if temp_dict.get(key) == None:
        temp_dict[key] = 1
    else:
        temp_dict[key] += 1
        
eval_dict = dict(sorted(temp_dict.items()))

In [ ]:
#training block for classical pipeline

train_set, test_set = pip.create_dataset(train_labels, train_circuits), pip.create_dataset(test_labels, test_circuits)
pip.create_trainer(train_circuits, test_circuits)
pip.train_model(train_set, test_set)
pip.plot() 

In [22]:
train_tensors = []
test_tensors = []

for circuit in train_circuits:
    symbols = set(circuit.free_symbols)
    symbol2index = {symbol: i for i, symbol in enumerate(symbols)}

    N = len(symbols)
    matrix = np.zeros((N, N), dtype=bool)
    for i, symbol_i in enumerate(symbols):
        for j, symbol_j in enumerate(symbols):
            if any(box.dom == symbol_i and box.cod == symbol_j for box in circuit.boxes):
                matrix[i, j] = True

    tensor = torch.tensor(np.stack([matrix, matrix], axis=-1), dtype=torch.bool)
    train_tensors.append(tensor)

for circuit in test_circuits:
    symbols = set(circuit.free_symbols)
    symbol2index = {symbol: i for i, symbol in enumerate(symbols)}

    N = len(symbols)
    matrix = np.zeros((N, N), dtype=bool)
    for i, symbol_i in enumerate(symbols):
        for j, symbol_j in enumerate(symbols):
            if any(box.dom == symbol_i and box.cod == symbol_j for box in circuit.boxes):
                matrix[i, j] = True

    tensor = torch.tensor(np.stack([matrix, matrix], axis=-1), dtype=torch.bool)
    test_tensors.append(tensor)

In [ ]:
from lambeq import BobcatParser, AtomicType
from lambeq import TensorAnsatz
from discopy import Box, Ty, Dim, Word
# Define atomic types
N = AtomicType.NOUN
S = AtomicType.SENTENCE

sentence = 'John walks in the park'

# Get a string diagram
parser = BobcatParser(verbose='text')
tensor_ansatz = TensorAnsatz({N: Dim(2), S: Dim(2)})
diagram = parser.sentence2diagram(sentence)
tensor_diagram = tensor_ansatz(diagram)


pad_diagrams = [tensor_ansatz(parser.sentence2diagram("parola")) for i in range(1, 20)]

new_diagram = tensor_diagram
for pad in pad_diagrams:
    new_diagram = pad @ new_diagram

print(new_diagram)
print(len(new_diagram))
print(len(tensor_diagram))
